In [ ]:
import json
import csv
import tweepy
import re
import pandas as pd
import os, glob
import plotly.express as px

## Authentication 

In [ ]:
# keys
consumer_key = 'PXLCJjdO4dcuW8v378ufCj3rC'
consumer_secret = 'CJu1Njasd1E9yscWEJe8oWxYPFr2wDECSuMRc68cfMygRjrUUI'
access_token = '1376138564321357826-Q5Cl52E7H8NC7Z7MbDBYYasnl156dR'
access_token_secret = 'TbJcAAI2FwQyem8ccPq2i98y7pS1ovI4msUtVk88R94Lf'

In [ ]:
# authorize twitter, initialize tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
  print("Error during authentication")

# Datasets

## Right-wing extremists

In [ ]:
right_wing = ['CIS_org', 'DustinInmanSoc', 'FAIRImmigration', 'StolenLivesQLT', 'AZ_Patriots', 'Crowdifornia','helpsavemd', 'IRLILaw','OregonIR','proenglishUSA',
              'ProtectRborders','AllianceDefends','ACPeds', 'AmericanFamAssc','AmericanVision','FridayFax','RJRushdoony','TXconcern','DJKMinistries','FRCdc',
              'family_watch','ProFamilyIFI','libertycounsel','MassResistance','PacificJustice','ProbeMinistries','PublicFreedom','RuthInstitute','savecalifornia','StedfastK',
              'StrongHoldBC','UFI','ProFamOrg','ACTforAmerica','AFAlliance','AFDINational','securefreedom','MartinMawyer','HorowitzCenter','GlobalFaithInst',
              'patriots_last','PoliticalIslam','gpatriotradio','usamaway','Thomas_More_Law','worldnetdaily','HistoryinReview','codohoutreach','901Confederate','identitydixie1',
              'nationvanguard','Dissentbooks','FolkAsatru','AsatruAFA','american_front','cathfamilynews','CultureWarsMag','fidelity_press','remnantnews','TraditionAction',
              'AFParty1','Identify_Evropa','AntelopeHill','NewRightAmerica','KosChertified','NatReformation','UpdatesPF','PatriotFront8','RADIXJournal_','redicetv',
              'TRSPodcasts','vdare','afpwebmaster','TheBarnesReview','RealistReport','ChickPub','JewishTaskForce','TonyAlamoMinist','TruNews', 'ALIPAC', 
              'rgarciaq', 'StephenM', 'MariaforAmerica', 'PeterLaBarbera', 'DrJamesDManning', 'sanderson1611', 'olivetree1569', 'tommymcmurtry2', 'realscottlively','pastortombrown', 
              'LivePrayer', 'ACTBrigitte', 'HogriderKafir', 'horowitz39', 'AmyMek', 'walidshoebat', 'TheUnitedWest', 'emperorInvictus', 'kylebristow', 'AprilintheNorth', 
              'RichardBSpencer', 'frankgaffney', 'GenBoykin', 'jihadwatchRS', 'RealJudgeMoore','christogenea', 'ABKreis3', 'bogritzUSA', 'ThomasRobb', 'drchuckbaldwin', 
              'Radomysisky', 'josephfarah', 'larryEKlayman', 'larrypratt', 'michaelboldin', 'IrvingBooks', 'rooshv', 'realCraigCobb', 'Gerhard_Lauck', 'kevin_a_strom', 
              'FrommPaul', 'KynanDutton', 'McnallenStephen', 'MattGaspers','ProfBMcCall', 'chesterdoles', 'TOOEdit', 'JamesEdwardsTPC', 'Church_Militant', 'johndenugentESA', 
              'ramzpaul', 'peterbrimelow', 'Tom_Sunic', 'TheMadDimension', 'JackPosobiec', 'Cernovich', 'NickJFuentes', 'Jeffrensei', 'DrAdrianHKrieg', 'BroNate1',
              'realRickWiles', 'TexasNatMov', 'ThisIsTexasFF']


## Left-wing extremists

In [ ]:
left_wing = ['IronFrontUSA', 'AntifaUPNY', 'AKPressDistro', 'deantifa', 'ash_antifa', 'afainatl', 'berkeleyantifa', 'HubCityAntifa', 'antifa_ne', 'NYCAntifa', 
             'RWParlerWatch', 'AntifaSac_', 'OCHittman', 'OrlandoAntifa', 'RoseCityAntifa', 'RadNewsMedia', 'TorchAntifa', 'UR_Ninja', 'BlackMastAntifa', 'communistsusa', 
             'antifa_news1', 'ALFcyberbrigade', 'veganzx1', 'redneckrevolt', 'SDRedneckRevolt', 'northcountryRR', 'LakeRevolt', 'SteelCityJBGC', 'PugetSoundJBGC', 'MidmoJBGC', 
             'SPKJBGC', 'BlackRiders1996', 'TheBLKPanthers', 'BlckPanthersOrg', 'PantherPartyWA', 'BPP510', 'TheRevo53655449', 'IUICOfficial', '1WEST_ISUPK', 'TheOfficialGra3', 
             'TheBlackRevolu3', 'followbamn', 'SraSwva', 'CentralPasra', 'FirearmsWap', 'DerbycityJ', 'PNWYLF', 'seattleYLF', 'EugYLF', 'NYCYLF', 
             'sw_ylf', 'chicagoylf', 'fla_ylf', 'TENNYLF', 'MarylandYLF', 'BellinghamYlf', 'CNYYLF', 'HONOYLF', 'ClarkYlf', 'ClarkFront', 
             'SouthernYLF', 'IGD_News', 'norcalylf', 'CarolinaYlf', 'screwstonafc', 'utah161', 'Haleyville_ARA', 'discord__panic', 'MNUnitedAF', 'WANaziWatch', 
             'HollerNetwork', 'LBCAFA', 'ara_losangeles', 'JulezDelphiki', 'avlantiracism', 'NoNazisAtUF', 'StruggaloCircus', 'florida_afa', 'no_nazi_auburn', 'ohshit_atx', 
             'ElmCityAntifa', 'antifa_knox', 'STLAntiFa', 'NorthNjAntifa', 'CoosAntifa', 'ashevilleantifa', 'PNWAWC', 'IndianaAntifa', 'AntifaDFW', 'CentralPAAntifa', 
             'downtownlesbos', 'antifaintl', 'Unitedsknhead', 'COSAntiFascists', 'aLoveraFighter1', 'ClevelandAntifa', 'SnitchWatch', 'PNWAntifascist', 'trashpandaAFA', 'Antifagator', 
             'antifascist45', 'nfa_antifascist', 'days_unity', 'LAAB213', 'CampusAntifNet', 'AntifaCowRustlr', 'DentonLeft', 'macc_nyc', 'IAF__FAI', 'IowaSRA', 
             'maine_sra', 'SRA_Minnesota', 'NEAlabamaSra', 'PugetSoundSRA', 'ChicagoSra', 'NYC_SocialistRA', 'SraAntonio', 'northgasra', 'SRAKentucky', 'NEO_SRA', 
             'reno_sra', 'sactownSRA', 'AnchorageSRA', 'TriadNCSRA', 'ArkansasSRA', 'EastLA_SRA', 'SRA_DFW', 'ScSra', 'pdxsra', 'PittsburghSra', 
             'PhillySRA', 'SocialistRA', 'HoustonSRA', 'newhampshireSRA', 'SRANewMexico', 'Nebraska_SRA', 'SanDiegoSRA', 'srahotv', 'latinorifleorg', 'MestizoLeftist', 
             'N_IL_SRA', 'SocialistRAMKE', 'TallahasseeSRA', 'TucsonSRA', 'UtahSRA', 'montana_sra', 'CorpusSra', 'Anarchistsl', 'AntifaGarfield', 'Central_FL_SRA',
             'FrontRangeSRA', 'PortlandSRA', 'iuicdenver', 'IUIC_Louisiana', 'IUICSEATTLE', 'IUICHawaii', 'IUICROCHESTER','IUICDetroit', 'IUICCorpus', 'IUIC_OHIO', 
             'iuic_kentucky', 'IUICMISSISSIPPI', 'IuicOrlando', 'IUIC_MIAMI', 'IUICNASHVILLE', 'IUICARKANSAS', 'IUICMEMPHIS', 'iuickansascity', 'iuicbirmingham', 'IUIC_Jville', 
             'IUICAtlantaCamp', 'iuicdallas', 'iuic_chicago', 'iuictallahassee', 'iuiclosangeles', 'iuicokc', 'Israelunitestl', 'IsupkN', 'IsupkG', 'IsupkB', 
             'isupkarizona', 'ISUPKDelaware2', 'IsupkHouston', 'IsupkOklahoma', 'IsupkAtlanta']


## Non-extremists

In [ ]:
non_extremists = ['SenWarren', 'KamalaHarris', 'CoryBooker', 'BernieSanders', 'maziehirono', 'EdMarkey', 'SenatorBaldwin', 'SenGillibrand', 'brianschatz', 'SenJeffMerkley', 
                  'PattyMurray', 'SenatorDurbin', 'SenStabenow', 'SenDuckworth', 'RonWyden', 'MartinHeinrich', 'SenatorCardin', 'SenBobCasey', 'SenatorCantwell', 'SenatorShaheen', 
                  'SenatorHassan', 'ChrisCoons', 'SenatorBennet', 'SenatorTester', 'MarkWarner', 'CaptMarkKelly', 'SenatorCarper', 'SenAngusKing', 'kyrstensinema', 'Sen_JoeManchin', 
                  'SenatorCollins', 'lisamurkowski', 'SenCapito', 'MittRomney', 'ChuckGrassley', 'SenJohnHoeven', 'senrobportman', 'SenatorWicker', 'SenKevinCramer', 'cindyhydesmith',
                  'SenJoniErnst', 'RogerMarshallMD', 'SenJohnBarrasso', 'JimInhofe', 'KLoeffler', 'SteveDaines', 'marcorubio', 'SenatorLankford', 'SenTomCotton', 'SenJohnKennedy', 
                 ]
len(non_extremists)

In [ ]:
non_extremists = ['SenToomey', 'HawleyMO', 'SenLummis', 'SenSasse', 'BillHagertyTN', 'SenatorBraun', 'SenTedCruz', 'RandPaul', 'SenMikeLee', 'TTuberville',
                 'taylorswift13', 'elonmusk', 'katyperry', 'ArianaGrande', 'BarackObama', 'BillGates', 'TheEllenShow', 'KimKardashian', 'ladygaga', 'JLo', 
                  'HillaryClinton', 'justinbieber', 'jtimberlake', 'KingJames', 'Eminem', 'MileyCyrus', 'Oprah', 'maddow', 'rihanna', 'seanhannity', 
                  'selenagomez', 'shakira', 'theweeknd', 'jimmyfallon', 'kanyewest', 'mariahcarey', 'ddlovato', 'AOC', 'Alyssa_Milano', 'AvrilLavigne', 
                  'chrisbrown', 'ConanOBrien', 'drake', 'TheRock', 'IGGYAZALEA', 'KevinHart4real', 'LeoDiCaprio', 'Pink', 'MichelleObama', 'Beyonce']
len(non_extremists)

## Create csv files

In [ ]:
# tweets from specific persons
# code from: https://gist.github.com/yanofsky/5436496

def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []  
    
    #make initial request for most recent tweets
    new_tweets = api.user_timeline(screen_name = screen_name, count=5000, tweet_mode='extended')
    
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print(f"getting tweets before {oldest}")
        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name, count=5000, tweet_mode='extended', max_id=oldest)
        
        #save most recent tweets
        alltweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        print(f"...{len(alltweets)} tweets downloaded so far")
    
    #transform the tweepy tweets into a 2D array that will populate the csv 
    outtweets = [[tweet.id_str, tweet.user.screen_name, tweet.created_at, tweet.user.description, 
                  tweet.full_text.encode("utf-8"),
                  tweet.retweet_count, tweet.favorite_count, tweet.lang] 
                 for tweet in alltweets]
    
    #write the csv  
    with open(f'data/{screen_name}_tweets.csv', 'wt', encoding="utf8") as f:
        writer = csv.writer(f)
        writer.writerow(["id", "user.screen_name", "created_at", "user.description",
                         "text","retweet_count", "favorite_count", "lang"])
        writer.writerows(outtweets)
    
    pass

if __name__ == '__main__':
    #pass in the username of the account you want to download
    for name in non_extremists:
        get_all_tweets(name)

## Left dataframe

In [ ]:
### left wing extremists
path = "data/leftwing"

all_files_left = glob.glob(os.path.join(path, "*.csv"))
#print(len(all_files_left))

# combine all files
combined_left = pd.concat([pd.read_csv(f, header=0) for f in all_files_left]) 
combined_left["label"] = "LWE"
combined_left.to_pickle("all_left_extremists.pkl")


In [ ]:
# read df_left to pkl
df_left = pd.read_pickle("all_left_extremists.pkl") # 302767x13
df_left.shape

# # show csv
# df_left.to_csv("all_left_extremists.csv") # 298226x12
# df_left_csv = pd.read_csv("all_left_extremists.csv")
# df_left_csv.shape


### Users

In [ ]:
df_left['user.screen_name'].nunique()

## Right dataframe

In [ ]:
## right wing extremists

path = "data/rightwing"
all_files_right = glob.glob(os.path.join(path, "*.csv"))
#print(len(all_files_right))

# combine all files
combined_right = pd.concat([pd.read_csv(f, header=0) for f in all_files_right])
combined_right["label"] = "RWE"
combined_right.to_pickle("all_right_extremists.pkl")


In [ ]:
# read df_right
df_right = pd.read_pickle("all_right_extremists.pkl") # 293060x9
df_right.shape

# combined_right.to_csv("all_right_extremists.csv")

# # show csv
# df_right.to_csv("all_right_extremists.csv") # 298226x12
# df_right_csv = pd.read_csv("all_right_extremists.csv")
# df_right_csv.shape


### Users

In [ ]:
df_right['user.screen_name'].nunique()

## Non-extremist Dataframe

In [ ]:
## non-extremist 

path = "data/nonextremists"
all_files_nonex = glob.glob(os.path.join(path, "*.csv"))
#print(len(all_files_nonex))

# combine all files
combined_nonex = pd.concat([pd.read_csv(f, header=0) for f in all_files_nonex])
combined_nonex["label"] = "NE"
combined_nonex.to_pickle("non_extremists.pkl")


In [ ]:
# read df_right
df_nonex = pd.read_pickle("non_extremists.pkl") # 295444
df_nonex.shape

# # show csv
# df_nonex.to_csv("non_extremists.csv") # 314271x13
# df_nonex_csv = pd.read_csv("non_extremists.csv")
# #df_nonex_csv.shape


In [ ]:
df_nonex.head()

In [ ]:
df_nonex['user.screen_name'].nunique() # 295041

## Both extremists

In [ ]:
# extremists = pd.concat([df_left, df_right]).reset_index(drop=True) # 586334, 12
# extremists.to_pickle("all_extremists.pkl")
# df_extremists = pd.read_pickle("all_extremists.pkl") 
# df_extremists.head()

# # create dataframe of both right-wing and left-wing extremists
# extremists = extremists.to_csv("all_extremists.csv")

In [ ]:
# df_extremists = pd.read_csv("all_extremists.csv")
# df_extremists = df_extremists.drop(['Unnamed: 0'], axis = 1)
# df_extremists # 586334x12

In [ ]:
extremists = pd.concat([df_left, df_right, df_nonex]).reset_index(drop=True) 
extremists.to_pickle("all_extremists.pkl")
df_extremists = pd.read_pickle("all_extremists.pkl") 
df_extremists # 891271

# create dataframe of both right-wing and left-wing extremists
# extremists = extremists.to_csv("all_extremists.csv")

# df_extremists = pd.read_csv("all_extremists.csv")
# df_extremists = df_extremists.drop(['Unnamed: 0'], axis = 1)
# df_extremists 

In [ ]:
df_extremists.groupby(['label'])['user.screen_name'].nunique()

In [ ]:
df_extremists.groupby(['label'])['text'].nunique() # 866515 tweets in total

In [ ]:
tweets = df_extremists.groupby('label')['text'].nunique().to_frame(name='count')
tweets['percentage'] = ((tweets['count'] / tweets['count'].sum()) * 100).round(1)
print(tweets)

fig = px.bar(tweets, y = "count", text="percentage", labels=dict(label="Extremist groups", count=""))
fig.update_layout(title="Percentage of tweets per group", title_x= 0.5, showlegend=False)
fig.show()

### Users

In [ ]:
df_extremists['user.screen_name'].nunique()